<a href="https://colab.research.google.com/github/divyani95/Transaction_details_Project/blob/main/Transaction_Data_Cleaned(VSCODE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import os


def read_and_clean_file():
    # Prompt user to select a file (Excel or CSV)
    file_path = filedialog.askopenfilename(
        title="Select Transaction File",
        filetypes=(("Excel files", "*.xls;*.xlsx"), ("CSV files", "*.csv"), ("All files", "*.*"))
    )

    # If no file is selected, exit the function
    if not file_path:
        messagebox.showinfo("No file selected", "Please select a file to process.")
        return

    try:
        # Read the raw file
        if file_path.lower().endswith('.csv'):
            df_raw = pd.read_csv(file_path, header=None)
        elif file_path.lower().endswith('.xls'):
            df_raw = pd.read_excel(file_path, header=None, engine='xlrd')
        else:
            df_raw = pd.read_excel(file_path, header=None, engine='openpyxl')

        # Identify the header row and reload with correct headers
        header_row = 14  # Assuming header is at row index 14
        if file_path.lower().endswith('.csv'):
            df = pd.read_csv(file_path, header=header_row)
        elif file_path.lower().endswith('.xls'):
            df = pd.read_excel(file_path, header=header_row, engine='xlrd')
        else:
            df = pd.read_excel(file_path, header=header_row, engine='openpyxl')

        # Drop "Unnamed: 0" column if it exists and is empty
        if "Unnamed: 0" in df.columns and df["Unnamed: 0"].isna().all():
            df.drop(columns=["Unnamed: 0"], inplace=True)

        # Define helper function for case-insensitive column matching
        def get_column_name(possible_names, df_columns):
            for name in df_columns:
                if any(possible_name.lower() == name.lower() for possible_name in possible_names):
                    return name
            return None

        # Identify the relevant columns
        transaction_date_col = get_column_name(["Transaction Date"], df.columns)
        debit_amount_col = get_column_name(["Debit Amount"], df.columns)
        credit_amount_col = get_column_name(["Credit Amount"], df.columns)
        running_balance_col = get_column_name(["Running Balance"], df.columns)
        description_col = get_column_name(["Transaction Description"], df.columns)

        # Drop "Credit Amount" and "Running Balance"
        df.drop(columns=[col for col in [credit_amount_col, running_balance_col] if col], inplace=True, errors="ignore")

        # Remove rows where "Debit Amount" is 0.00 or "0.00" (as a string)
        if debit_amount_col:
           # df[debit_amount_col] = pd.to_numeric(df[debit_amount_col], errors="coerce")  # Convert to float
           # df = df[df[debit_amount_col].astype(float) != 0.00]
            df = df[df[debit_amount_col].notna()]  # Remove NaN values (if conversion failed)
            df = df[~(df[debit_amount_col] == 0.00)]
        # Keep only transactions where "Transaction Description" starts with "NEFT" or "RTGS"
        if description_col:
            df = df[df[description_col].str.startswith(("NEFT", "RTGS"), na=False)]
        else:
            messagebox.showwarning("Column Missing", "'Transaction Description' column not found.")

        # Split "Transaction Date" into "Transaction-Date" & "Transaction-Time"
        if transaction_date_col:
            df[["Transaction-Date", "Transaction-Time"]] = df[transaction_date_col].astype(str).str.split(" ", expand=True)
            df.drop(columns=[transaction_date_col], inplace=True)

        # Convert "Transaction-Date" to proper date format
        df["Transaction-Date"] = pd.to_datetime(df["Transaction-Date"], errors="coerce").dt.date

        # Convert "Transaction-Time" to proper time format if available
        if "Transaction-Time" in df.columns:
            df["Transaction-Time"] = pd.to_datetime(df["Transaction-Time"], errors="coerce").dt.time

        # Save the cleaned dataset
        output_path = os.path.join(os.path.dirname(file_path), "Transactions_Details_Cleaned.xlsx")
        df.to_excel(output_path, index=False, engine='openpyxl')

        # Show success message
        messagebox.showinfo("Success", f"Cleaned dataset saved at:\n{output_path}")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred:\n{e}")


def create_gui():
    # Create the main application window
    root = tk.Tk()
    root.title("Transaction Data Cleaner")

    # Set a minimum size for the window
    root.minsize(300, 150)

    # Create a button to read and clean the file
    read_button = tk.Button(root, text="Read File", command=read_and_clean_file, width=20, height=2)
    read_button.pack(pady=40)

    # Start the GUI event loop
    root.mainloop()


if __name__ == "__main__":
    create_gui()
